## Run cells sequentially:

In [2]:
import cv2
import imutils
import matplotlib
import numpy as np

img=cv2.imread('/scratch/user/r.kapoor/compete/ECEN649_Project_files/dataset/trainset/faces/face00001.png',
               cv2.IMREAD_GRAYSCALE)
scale=3
w = int(img.shape[1] / scale)
img2 = np.asarray(imutils.resize(img, width=w),dtype="int64")

In [3]:
detectorWidth=19
scale=1
w = detectorWidth#int(img.shape[1] / scale)
img = np.asarray(imutils.resize(img, width=w),dtype="int64")

In [4]:
#Integral Image
Matrix_S=np.reshape([np.sum(img[range(j+1),:],axis=0) for j in range(detectorWidth)],(detectorWidth,detectorWidth))
Int_image=np.transpose(np.reshape([np.sum(Matrix_S[:,range(j+1)],axis=1) 
                                   for j in range(detectorWidth)],(detectorWidth,detectorWidth)))

In [5]:
#Integral Image access function
def getIntImageValues(Int_image, index_i,index_j):
            index_valid = (index_i>=0)*(index_j>=0)
            index_i_valid=(index_valid*index_i).astype(int)
            index_j_valid=(index_valid*index_j).astype(int)
            return Int_image[index_i_valid,index_j_valid]-Int_image[0,0]*np.invert(index_valid)
A=getIntImageValues

In [6]:
def getFeature1_vertices(detectorWidth, haar_j_len,haar_k_len):
    Bottom_j=[ [j,j-np.int(haar_j_len/2)+1] for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    Bottom_up_k=[[k,k+haar_k_len-1]
              for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    Top_j=[ [j-np.int(haar_j_len/2),j-haar_j_len+1] for j in range(haar_j_len-1,detectorWidth) 
              for k in range(detectorWidth-haar_k_len+1)]
    #np.array( [j,k] for j in Up_j  for k in Up_k )
    return  (Bottom_j,Bottom_up_k,Top_j)

In [7]:
#Extremes of covered area
def k_min(a):
    """Min value of j for vertex (i,j) within haar feature 1"""
    return a[0]
def k_max(a):
    """Max value of j for vertex (i,j) within haar feature 1"""
    return a[1]
def j_min(a):
    """Min value of i for vertex (i,j) within haar feature 1"""
    return a[1]
def j_max(a):
    """Max value of i for vertex (i,j) within haar feature 1"""
    return a[0]

In [8]:
#detectorWidth=19
haar_J_len=np.arange(2,2*int(detectorWidth/2)+1,2)
haar_K_len=np.arange(1,detectorWidth+1)
num_features_subtypes=len([(m,n) for m in haar_J_len for n in haar_K_len])

In [9]:
def feature1_IntImage_vertices(detectorWidth, haar_j_len,haar_k_len):
    Bottom_j,Bottom_up_k,Top_j =getFeature1_vertices(detectorWidth, haar_j_len,haar_k_len)
    K_min=np.apply_along_axis(k_min,1,Bottom_up_k).astype(int)
    K_max=np.apply_along_axis(k_max,1,Bottom_up_k).astype(int)
    J_min=np.apply_along_axis(j_min,1,Bottom_j).astype(int)
    J_max=np.apply_along_axis(j_max,1,Bottom_j).astype(int)
    J_min_up =np.apply_along_axis(j_min,1,Top_j).astype(int)
    J_max_up =np.apply_along_axis(j_max,1,Top_j).astype(int)
    return (K_min,K_max,J_min,J_max,J_min_up,J_max_up)

In [10]:
my_result=[feature1_IntImage_vertices(detectorWidth, m,n)\
for m in haar_J_len for n in haar_K_len]

In [11]:
K_min=np.array([]).astype(int)
K_max=np.array([]).astype(int)
J_min=np.array([]).astype(int)
J_max=np.array([]).astype(int)
J_min_up =np.array([]).astype(int)
J_max_up =np.array([]).astype(int)
for j in range(num_features_subtypes):
    K_min = np.append(K_min,my_result[j][0]).astype(int)
    K_max = np.append(K_max,my_result[j][1]).astype(int)
    J_min = np.append(J_min,my_result[j][2]).astype(int)
    J_max = np.append(J_max,my_result[j][3]).astype(int)
    J_min_up = np.append(J_min_up,my_result[j][4]).astype(int)
    J_max_up = np.append(J_max_up,my_result[j][5]).astype(int)

In [12]:
import pandas as pd
Feature1_indices = pd.DataFrame({ 'K_min': K_min, 'K_max': K_max,\
                                'J_min':J_min, 'J_max':J_max,\
                                'J_min_up':J_min_up, 'J_max_up':J_max_up})
Feature1_indices.to_csv("Int_image_indices/Feature1_indices.csv")

In [13]:
def getFeature1Values(Int_image, K_min,K_max,J_min,J_max,J_min_up,J_max_up):
            Bottom_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            Top_area=A(Int_image,J_max_up,K_max)-A(Int_image,J_max_up,K_min-1)-\
A(Int_image,J_min_up-1,K_max)+A(Int_image,J_min_up-1,K_min-1)
            return Bottom_area-Top_area

In [14]:
Feature1=getFeature1Values(Int_image, K_min,K_max,J_min,J_max,J_min_up,J_max_up)

In [15]:
Feature1.shape

(17100,)